In [1]:
from dotenv import load_dotenv
from operator import itemgetter
from examples import few_shot_examples
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ChatMessageHistory

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
history = ChatMessageHistory()
vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    few_shot_examples,
    OpenAIEmbeddings(),  
    vectorstore,
    k=5,
    input_keys=["input"],
)

In [2]:
sqlite_uri = 'sqlite:///Excel and databases/Data3.db' 
db = SQLDatabase.from_uri(sqlite_uri)
# db.dialect
# db.get_usable_table_names()
# db.run("SELECT * FROM employees;")
# db.run("SELECT * FROM perks;")
# db.run("SELECT * FROM benefits;")
# db.run("SELECT * FROM insurance;")

In [3]:
question="Hi"
generate_query = create_sql_query_chain(llm, db)
sql_query = generate_query.invoke({"question": question})
print(sql_query)

SELECT date('now');


In [4]:
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(sql_query)

chain = generate_query | execute_query

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user questiony.Your answer should contain thank you at the end of messages.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below.

In [5]:
rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)
chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers

In [6]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
example_prompt

ChatPromptTemplate(input_variables=['input', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}\nSQLQuery:')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])

In [7]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input", "top_k"],
)
few_shot_prompt

FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002DBADB54EE0>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), input_variables=['input', 'top_k'], example_prompt=ChatPromptTemplate(input_variables=['input', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}\nSQLQuery:')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]))

In [32]:
import time
from datetime import datetime

user = "Alex"
gender = "Male"
designation = "GM"
current_time = time.time()
current_date = datetime.utcfromtimestamp(current_time).strftime('%Y-%B-%d')
print(current_date)

2024-May-08


In [40]:
system_message = f"""
Your name is Hira, friendly HR of Meezan Bank of Pakistan. You are responsible to solve {user} queries only which relates to the {user} information.
Your task is to identify if the user wants to see the leave data or applying for a leave after analyzing the whole conversation.
The current date is {current_date}.
You have to three types of leave data available in your database (Annual, Casual, Sick).
Make sure users can only select previous dates for sick leave. The dates for annual and casual leave should be after {current_date} 
You are designed to interact with a SQL database specifically for {user}, {gender}, and {designation}.
""" + """
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
""" + f"""
Don't give false information.
If a user wants to chat with you then generate response accordingly. 
You're allowed to talk with the user in a friendly way.
You're allowed to give only {user} information and nothing else.
Make sure if {user} ask anything related to the database then simply apologize in a professional tone.
If a {user} asking anyone's details present in the database, just return "You are not authorized to access employee details. You may only access your own information.".

Stick to this professional tone and engage the user by asking other leave related question.

In this part, you have only conern with leaves data of you have only three types of leaves ( Annual, Casual, Sick ).  \n
regarding leaves, {user} can ask two type of queries ( want to see leaves data or apply for a leaves ). If you are confuse or not confirm \n
that whether a {user} want to see leaves data or apply then this is your responsibility to confirm first then proceed next. 
Do not show the all information in a single prompt. you must follow the steps. below are some infomation about steps.

You must confirm the type of leaves first then show only this leave type information.
- If {user} want to see leaves data, then you must follow below steps for it.
    1- first ask the type of the leave, ( Annual, Casual, Sick ) .
    2- If user select 'sick' or ask, then show 'sick data' only with some prayer message of the {user}.
    3- If user select 'casual', then show only 'casual data' only, with asking in a friendly tone that 'are you planning for some holiday? or this type \n
       of other question'.
    4- If user select 'annual', then show only 'annual data' only, with asking in a friendly tone that 'are you planning for some vacation with family? or \n
        this type of other question'.

- If {user} wants to apply for leaves, then you must follow below steps for it.
    1- first ask the type of the leave, ( Annual, Casual, Sick ) .
    2- If user select 'sick', then show 'sick data' only with some prayer message for the {user}. Also ask for a dates for taking/applying a 'sick leaves'.
        2.1- When user tell you the dates, then tell her/him, 'Can I write a sick leave application for you according to your given dates?'. If {user} say 'yes' 
            ,then application according to the given information. Make sure {user} can only select previous dates for sick leave.
    3- If user select 'casual', then show only 'casual data' only, with asking in a friendly tone that 'are you planning for some holiday? or \n
        this type of other question'. also ask for a dates for taking/applying a 'casual leaves'
        3.1- When user tell you the dates, then check the given dates and make sure it should not fall in the blackout dates. Make sure the dates for casual leave should be after {current_date}.
        3.2- If the given dates are fall/present in a blackout dates, then apologize to {user} and inform her/him that the given dates are falling in the blackout \n
            dates along with the reason and suggest him/her some months/dates that are not fall in blackouts or tell him/her to select other dates. \n
        3.3- If the given dates are not fall in blackout dates then tell him/her happily that the given dates are available with reason( because \n
            the given dates are not falling in blackout dates) for you to take a leave, and suddenly ask then 'Can I write a casual leave application for you with your given dates?'. If {user} \n
            say 'yes', then type application according to the given information.	
    4- If user select 'annual' , then show only annual data' only, with asking in a friendly tone that 'are you planning for some vacation? or \n
        this type of other question'. also ask for a dates for taking/applying a annual leaves'
        4.1- When user tell you the dates, then check the given dates and make sure it should not fall in the blackout dates. Make sure the dates for annual leave should be after {current_date}.
        4.2- If the given dates are fall/present in a blackout dates, then apologize to {user} and inform her/him that 'the given dates are falling in the blackout \n
            dates along with the reason' and suggest him/her some months/dates that are not fall in blackouts or tell him/her to select other dates. \n
        4.3- If the given dates are not fall in blackout dates then tell him/her happily that the given dates are available with reason( because \n
            the given dates are not falling in blackout dates) for you to take a leave, and suddenly ask then 'Can I write annual leave application for you along with your given dates?'. If {user} \n
            say 'yes', then type application according to the given information.
        4.4 - Here are the blackout dates that must not coincide with your leave requests:
                - December 1st, 2024 to December 10th, 2024 (due to audit period)
                - January 1st, 2024 to January 10th, 2024 (due to system maintenance)
                - February 1st, 2025 to February 10th, 2025 (business season).

Make sure you have asked all the questions that mentioned above before proceeding further queries.
"""

In [41]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message +
         """Here is the relevant table info: {table_info}.
            Maintain a professional tone and provide precise, to-the-point answers.
            Below are a number of examples of questions and their corresponding SQL queries. 
        """),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)


In [42]:
generate_query = create_sql_query_chain(llm, db, final_prompt)
generate_query

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
| ChatPromptTemplate(input_variables=['input', 'messages', 'table_info'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'top_k': '5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['table_info'], template='\nYour name is Hira, friendly HR of Meezan Bank of Pakistan. You are responsible to solve Alex queries only which relates to the Alex information.\nYour task is to identify if the user wants to see the leave data or applying for a leave after analyzing the whole conversation.\n

In [43]:
chain = (
# RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
         | ChatPromptTemplate(input_variables=['input', 'messages', 'table_info'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'top_k': '5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['table_info'], template='\nYour name is Hira, friendly HR of Meezan Bank of Pakistan. You are responsible to solve Alex queries only which relates to the Alex information.\nYour task is to identify if the user wants to see th

In [44]:
def langchain_query_executor(query):
    response = chain.invoke({"question": query, 
                             "messages": history.messages})
    history.add_user_message(query)
    history.add_ai_message(response)
    print(history.messages)
    return response

In [45]:
input_message = "Tell me my perks and benefits"
output = langchain_query_executor(input_message)
print("---------------------------")
print("---------------------------")
print(output)

[HumanMessage(content='Tell me my perks and benefits'), AIMessage(content="I'm sorry, but you are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Hi'), AIMessage(content="I'm sorry, but there seems to be a syntax error in the SQL query. Can you please provide more information or clarify your question? Thank you."), HumanMessage(content='Who are you?'), AIMessage(content='I am a language model AI designed to assist you with your queries. How can I help you today? Thank you.'), HumanMessage(content='What was last question that I asked you?'), AIMessage(content='I apologize, but I am not able to access your previous questions. How can I assist you further today? Thank you.'), HumanMessage(content="I'm Hassan. Tell me my perks and benefits"), AIMessage(content="I'm sorry, Hassan. You are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Who am I?')

In [46]:
input_message = "Who are you?"
output = langchain_query_executor(input_message)
print("---------------------------")
print("---------------------------")
print(output)

[HumanMessage(content='Tell me my perks and benefits'), AIMessage(content="I'm sorry, but you are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Hi'), AIMessage(content="I'm sorry, but there seems to be a syntax error in the SQL query. Can you please provide more information or clarify your question? Thank you."), HumanMessage(content='Who are you?'), AIMessage(content='I am a language model AI designed to assist you with your queries. How can I help you today? Thank you.'), HumanMessage(content='What was last question that I asked you?'), AIMessage(content='I apologize, but I am not able to access your previous questions. How can I assist you further today? Thank you.'), HumanMessage(content="I'm Hassan. Tell me my perks and benefits"), AIMessage(content="I'm sorry, Hassan. You are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Who am I?')

In [47]:
input_message = "I'm Hassan by the way."
output = langchain_query_executor(input_message)
print("---------------------------")
print("---------------------------")
print(output)

[HumanMessage(content='Tell me my perks and benefits'), AIMessage(content="I'm sorry, but you are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Hi'), AIMessage(content="I'm sorry, but there seems to be a syntax error in the SQL query. Can you please provide more information or clarify your question? Thank you."), HumanMessage(content='Who are you?'), AIMessage(content='I am a language model AI designed to assist you with your queries. How can I help you today? Thank you.'), HumanMessage(content='What was last question that I asked you?'), AIMessage(content='I apologize, but I am not able to access your previous questions. How can I assist you further today? Thank you.'), HumanMessage(content="I'm Hassan. Tell me my perks and benefits"), AIMessage(content="I'm sorry, Hassan. You are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Who am I?')

In [48]:
input_message = "What's my name?"
output = langchain_query_executor(input_message)
print("---------------------------")
print("---------------------------")
print(output)

[HumanMessage(content='Tell me my perks and benefits'), AIMessage(content="I'm sorry, but you are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Hi'), AIMessage(content="I'm sorry, but there seems to be a syntax error in the SQL query. Can you please provide more information or clarify your question? Thank you."), HumanMessage(content='Who are you?'), AIMessage(content='I am a language model AI designed to assist you with your queries. How can I help you today? Thank you.'), HumanMessage(content='What was last question that I asked you?'), AIMessage(content='I apologize, but I am not able to access your previous questions. How can I assist you further today? Thank you.'), HumanMessage(content="I'm Hassan. Tell me my perks and benefits"), AIMessage(content="I'm sorry, Hassan. You are not authorized to access employee details. You may only access your own information. Thank you."), HumanMessage(content='Who am I?')